In [1]:
#!/usr/bin/env python3
# Software License Agreement (BSD License)
#
# Copyright (c) 2121, UFACTORY, Inc.
# All rights reserved.
#
# Author: Vinman <vinman.wen@ufactory.cc> <vinman.cub@gmail.com>

"""
Description: joint velocity control
"""

import os
import sys
import time


from xarm.wrapper import XArmAPI



arm = XArmAPI("192.168.1.159")
arm.motion_enable(enable=True)
arm.set_mode(0)
arm.set_state(state=0)
time.sleep(1)

arm.reset(wait=True)

# set joint velocity control mode
arm.set_mode(4)
arm.set_state(0)
time.sleep(1)

count = 6
while count > 0:
    count -= 1
    arm.vc_set_joint_velocity([-50, 0, 0, 0, 0, 0])
    time.sleep(2)
    arm.vc_set_joint_velocity([50, 0, 0, 0, 0, 0])
    time.sleep(2)

# stop
arm.vc_set_joint_velocity([0, 0, 0, 0, 0, 0])

arm.disconnect()

SDK_VERSION: 1.13.14
ROBOT_IP: 192.168.1.159, VERSION: v1.12.2, PROTOCOL: V1, DETAIL: 6,9,LI1006,DL1000,v1.12.2, TYPE1300: [0, 0]
change protocol identifier to 3
[set_state], xArm is ready to move
